# 1. Uso de modelos de chat con LangChain

Un modelo de chat es una variante especializada de los modelos de lenguaje, diseñado y optimizado para manejar interacciones conversacionales. Estos modelos son el núcleo detrás de aplicaciones como los chatbots, asistentes virtuales y cualquier otra aplicación que requiera interacción en lenguaje natural.

En este ámbito, los modelos de chat trabajan con tres tipos de mensajes distintos:

1. `SystemMessage`: Este tipo de mensaje establece el comportamiento y los objetivos del Modelo de Lenguaje Masivos (LLM, por sus siglas en inglés). En `SystemMessage` se pueden plasmar instrucciones específicas tales como "Actuar como un gerente de marketing" o "Devolver solo una respuesta JSON sin texto explicativo".

2. `HumanMessage`: Este es el espacio donde se ingresan las instrucciones o consultas del usuario que posteriormente serán enviadas al LLM.

3. `AIMessage`: Aquí es donde se almacenan las respuestas generadas por el LLM. Este tipo de mensaje es importante para conservar el historial de chat, que luego será utilizado para enviar al LLM en futuras solicitudes y así mantener el contexto de la conversación.

Existe también un tipo genérico de mensaje denominado `ChatMessage`, que acepta una entrada de "rol" arbitraria y puede ser utilizado en circunstancias que requieran roles distintos a los tres mencionados previamente (System, Human, AI). Sin embargo, en la mayoría de los casos, se utilizan principalmente los tres tipos de mensajes mencionados anteriormente.


In [ ]:
%%capture
!pip install openai langchain

In [ ]:
from getpass import getpass
import os

OPENAI_API_KEY = getpass('Enter the secret value: ')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

Enter the secret value: ··········


In [ ]:
from langchain.chat_models import ChatOpenAI

chat_gpt3_5 = ChatOpenAI(
    temperature=0,
    model='gpt-3.5-turbo'
)

In [ ]:
from langchain.schema import (
    SystemMessage(content="Eres un asistente en un Call Center de reparación de lavadoras."),
    HumanMessage(content="Cómo estás? Necesito ayuda."),
    AIMessage(content="Estoy bien, gracias. En qué puedo ayudar?"),
    HumanMessage(content="Quiero reparar mi lavadora.")
)



En sí mismo, res es un AIMessage.

Podemos seguir el chat...

In [ ]:
# Append el res a nuestra serie de mensajes


# Agregamos un nuevo mensaje del humano


# send to chat-gpt


## 1.1 Chat prompt templates

Las plantillas de prompts en LangChain proporcionan una forma flexible y dinámica de interactuar con los LLM. En lugar de codificar prompts estáticos, estas plantillas permiten la incorporación de entradas variables del usuario.


In [ ]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

plantilla_sistema="""
Eres un experto en productos que debe proporcionar información detallada sobre productos de la marca {marca}.
"""



# 2. Memoria en LangChain

Comprender cómo se utiliza la memoria en LangChain es fundamental para construir chat efectivos. Su papel vital está en cómo se puede emplear para hacer que tus modelos sean más conversacionales y parecidos a los humanos.

## 2.1 La importancia de la memoria

A menudo se espera que los modelos de lenguaje interactúen como un conversador humano. Los usuarios asumen que el modelo recordará partes anteriores de la conversación, entenderá el contexto y responderá en consecuencia. Sin embargo, los grandes modelos de lenguaje como GPT-3 o GPT-4, en su esencia, son sin estado, es decir, no poseen una memoria inherente.

La memoria se vuelve vital en casos en los que un usuario se refiere a temas discutidos anteriormente o utiliza pronombres para referirse a una entidad mencionada anteriormente. Este fenómeno se conoce como _resolución de correferencia_ y es un desafío clave en el procesamiento del lenguaje natural.

```python
Ejemplo:
Usuario: "Conocí a un chico llamado John ayer. Él es futbolista."
```

Aquí, "Él" se refiere a "John". El modelo necesita memoria para resolver esta referencia.

## 2.2 Enfoques para la gestión de la memoria

Hay dos enfoques principales para la gestión de la memoria en LangChain:

1. Incorporar la memoria en la indicación.
2. Utilizar un mecanismo de búsqueda externo.

Profundizaremos en ambos enfoques en las siguientes secciones.

### 2.2.1 Incorporar la memoria en la indicación

La forma más sencilla de incorporar la memoria es incluir el historial de conversación en la indicación. Por ejemplo, consideremos una conversación con un asistente digital llamado 'Kate'.

```python
Contexto: Eres un asistente digital al que le gusta tener conversaciones llamado Kate.
Usuario: Hola, soy Sam.
Agente: Hola Sam, ¿cómo estás?
Usuario: Bien, ¿cuál es tu nombre?
Agente: Mi nombre es Kate.
Usuario: ¿Cómo estás hoy, Kate?
```
Este método implica agregar el historial de conversación a la indicación. El historial de conversación actúa como contexto para el modelo, permitiéndole comprender la discusión en curso. Sin embargo, este método tiene limitaciones. Dado que los modelos de lenguaje actuales como GPT-4 tienen un límite de tokens (por ejemplo, 4096 tokens para GPT-4), las conversaciones extensas no cabrían en este límite.

### 2.2.2 Búsqueda externa

Una alternativa a incrustar la memoria en la indicación es utilizar una búsqueda externa, como una base de datos o un grafo de conocimiento. Este método, aunque más complejo, puede manejar conversaciones más grandes y dinámicas que cubriremos en futuros capítulos.

## 2.3 Estrategias de gestión de memoria integradas en LangChain

LangChain ofrece estrategias integradas para gestionar la memoria de manera efectiva:

1. Inserción directa en la indicación: Como se discutió en la sección anterior, esta estrategia simplemente agrega el historial de conversación a la indicación.
2. Resumen de la conversación: Este método implica generar un resumen de la conversación e incluirlo en la indicación. El resumen en sí se realiza mediante un modelo de lenguaje separado.
2. Memoria de ventana: Aquí, solo se incluyen los últimos intercambios de la conversación en la indicación.
4. Mezcla de estrategias: Esta estrategia involucra una combinación de los últimos intercambios y una versión resumida del resto de la conversación.

Además, LangChain te permite personalizar estas estrategias o incluso implementar las tuyas propias, brindándote flexibilidad según los requisitos únicos de tu caso de uso.

Recuerda, la gestión de la memoria es una parte clave en la creación de modelos interactivos y atractivos. La estrategia que elijas puede afectar significativamente el rendimiento del modelo y la experiencia del usuario. Elige sabiamente y no temas experimentar e iterar para encontrar el enfoque más adecuado.


## 2.4 ConversationBufferMemory: conversaciones cortas y el enfoque ingenuo

Vamos a sumergirnos en la implementación práctica de la memoria en LangChain. Primero se debe configurar el entorno mediante la instalación de paquetes necesarios como OpenAI y LangChain.

In [ ]:
%%capture
!pip -q install openai langchain

In [ ]:
from getpass import getpass
import os

os.environ['OPENAI_API_KEY'] = getpass('Enter the secret value: ')

Enter the secret value: ··········


En primer lugar, nos centraremos en el tipo de memoria más sencillo, la "Memoria del Búfer de Conversación" (Conversation Buffer Memory, en inglés). Este tipo de memoria almacena la historia de la conversación directamente en el prompt, a medida que la conversación progresa.

En LangChain, el proceso de implementación es sencillo. Primero importamos el tipo de memoria de LangChain y lo instanciamos. Luego, pasamos esta instancia de memoria a nuestro modelo de lenguaje 'chain'.

Usaremos una simple "ConversationChain" para este ejemplo. Esta cadena nos permite conversar con un modelo llamando a la función 'conversation_predict' y pasando nuestro texto de entrada.

In [ ]:
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain import OpenAI
from langchain.chains import ConversationChain

In [ ]:
llm = OpenAI(model_name='text-davinci-003',
             temperature=0,
             max_tokens = 256,
             )

En el ejemplo anterior, hemos establecido `verbose=True`. Esto nos permite ver el mensaje completo y la respuesta del modelo cada vez que hacemos una predicción. El mensaje consiste en la conversación actual. Para la primera predicción, es simplemente "Hola, soy Sam", y la IA responde en consecuencia.

A medida que la conversación continúa, es importante tener en cuenta que nuestro búfer de conversación va acumulando nuestro historial conversacional. Lo que ocurre es que el mensaje que se introduce en el modelo de lenguaje se amplía con cada turno de la conversación. Este método funciona bien para conversaciones cortas, pero para diálogos extensos, puede llegar a ser demasiado largo para caber en el límite de tokens del modelo de lenguaje. Esta es una limitación que discutiremos a continuación.

Este enfoque de memoria intermedia de conversación, la forma más simple de memoria en LangChain, es sorprendentemente eficaz. Es particularmente adecuado para escenarios donde hay un número predeterminado de interacciones con el bot, o lo has programado de tal manera que después de un cierto número de turnos (digamos cinco), la conversación termina. En estos casos, la memoria intermedia de conversación servirá perfectamente a tus propósitos.

## 2.5 ConversationBufferWindowMemory - ¿Cuántas interacciones debemos recordar?

El sistema de `Memoria de Ventana del Búfer de Conversación`(`Conversation Buffer Window Memory`, en inglés) es algo similar a la `Memoria del Búfer de Conversación`, pero solo mantiene las últimas 'k' interacciones en la indicación. El parámetro 'k' se puede ajustar según tus preferencias y restricciones presupuestarias.

In [ ]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [ ]:
conversation.predict(input="Que ondi, como etai? Soy Omar y escribo muy coloquial.")

In [ ]:
conversation.predict(input="Estoy buscando que me hables coloquialmente pues hablar formal es aprehender mi libertad de expresión.")

In [ ]:
conversation.predict(input="Sobre la libertad de mi pueblo latinoamericano")

In [ ]:
conversation.predict(input="Que no estoy haciendo lo suficiente.")

In [ ]:
conversation.predict(input="¿Cómo me gusta escribir? Coloquial? O formal?")

The memory buffer shows every interaction no matter the k.

Este enfoque de gestión de memoria alimenta únicamente las últimas `k` interacciones en el Modelo de Lenguaje Grande, lo cual puede ser una táctica beneficiosa al interactuar con el bot. Por ejemplo, si estableces `k=5`, la mayoría de las conversaciones no requerirán hacer referencia a partes anteriores de la conversación. Es posible dar la impresión de una memoria a largo plazo a los usuarios simplemente manteniendo un recuerdo de los últimos tres a cinco pasos de la conversación.

Sin embargo, aunque la Memoria de Ventana del Búfer de Conversación solo suministra al Modelo de Lenguaje Grande las últimas 'k' interacciones, aún conserva toda la conversación. Esto es útil para inspeccionar el historial de la conversación o almacenarlo para futura referencia.

## 2.6 ConversationSummaryMemory: Crea un resumen de nuestras interacciones

Otro tipo de gestión de memoria en LangChain es la `Memoria de Resumen de Conversación` (`Conversation Summary Memory`, en inglés). La distinción clave aquí es que en lugar de mantener toda la conversación, LangChain la resume.

In [ ]:
from langchain.chains.conversation.memory import ConversationSummaryMemory
from langchain import OpenAI
from langchain.chains import ConversationChain

In [ ]:
llm = OpenAI(model_name='text-davinci-003',
             temperature=0,
             max_tokens = 256,)

Ahora, en lugar de transferir cada ida y vuelta entre el usuario y el bot al prompt, el sistema genera un resumen. Observa que, en esta fase, la versión resumida utiliza más tokens que la conversación sin procesar. Sin embargo, a medida que la conversación continúa, el resumen se vuelve más eficiente en cuanto a tokens.

In [ ]:
conversation.predict(input="Que ondi, como etai? Soy Omar y escribo muy coloquial.")

In [ ]:
conversation.predict(input="Estoy muy bien. Cuéntame sobre la revolución de las naciones latinoamericanas y la historia de la opresión indígena.")

In [ ]:
conversation.predict(input="Entonces por qué sigue habiendo racismo en la región? Habla coloquial para poderme identificar contigo.")

El resultado proporciona una versión resumida de la conversación hasta ahora, es decir, el humano (Omar) se presenta, pregunta cómo está el AI y solicita ayuda con el soporte al cliente. El AI (llamado AI) saluda a Sam y pregunta qué lo trae aquí hoy. En respuesta a la solicitud de Sam, el AI está dispuesto a ayudar y pregunta qué tipo de soporte se necesita.

Es importante tener en cuenta que el sistema de resumen mantiene una referencia constante a 'AI' y 'Omar', en lugar de usar pronombres como 'él', 'ella' o 'ello'. Este enfoque asegura claridad y ayuda a evitar posibles confusiones debido a la interpretación errónea de los pronombres.

## 2.7 ConversationSummaryBufferMemory - Crea un resumen a partir de cierto número de interacciones pasadas

El cuarto tipo de memoria es la `Memoria de Ventana de Resumen del Búfer` (`Summary Buffer Window Memory`, en inglés). Esta estrategia de memoria es una combinación de los dos primeros tipos que hemos visto, ya que incluye un aspecto de resumen y también mantiene un búfer de un número fijo de tokens. Esta estrategia dual permite equilibrar entre mantener un resumen completo de la conversación y reducir el uso de tokens para un funcionamiento más eficiente.


In [ ]:
from langchain.chains.conversation.memory import ConversationSummaryBufferMemory

In [ ]:
from langchain.chat_models import ChatOpenAI

llm = OpenAI(model_name='text-davinci-003',
             temperature=0,
             max_tokens = 512,
            verbose=True)

In [ ]:
%%capture
!pip install tiktoken

In [ ]:
from langchain import OpenAI

# Setting k=2, will only keep the last 2 interactions in memory
# max_token_limit=40 - token limits needs transformers installed
memory = ConversationSummaryBufferMemory(llm=OpenAI(), max_token_limit=40, k=2)

In [ ]:
conversation_with_summary = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

Las interacciones iniciales de la conversación seguirán un patrón similar al anterior. Sin embargo, una vez que la interacción supera el límite del búfer (es decir, el total de tokens supera el límite de 40 tokens), la memoria empieza a resumir las primeras interacciones, mientras mantiene el texto completo de las interacciones recientes.

In [ ]:
conversation_with_summary.predict(input="Que ondi, como etai? Soy Omar y escribo muy coloquial.")

Además, conserva el texto completo de las interacciones recientes hasta que el recuento total de tokens alcance nuevamente el límite. En este punto, la parte más temprana de la interacción se resume aún más o se elimina, y el proceso continúa. Esto permite que el bot conserve las partes más relevantes y recientes de la conversación en detalle completo, al tiempo que tiene un contexto resumido de las partes anteriores.

In [ ]:
conversation_with_summary.predict(input="Estoy muy bien. Cuéntame sobre la revolución de las naciones latinoamericanas y la historia de la opresión indígena.")

In [ ]:
conversation_with_summary.predict(input="Entonces por qué sigue habiendo racismo en la región? Habla coloquial para poderme identificar contigo.")

In [ ]:
conversation_with_summary.predict(input="¿Cuál fue la primera pregunta que te hice?")

### Prompts para cadenas con memoria

Podemos examinar el prompt que está recibiendo nuestra cadena por default y alterarlo a nuestro gusto.

In [ ]:
conversation_with_summary.prompt

In [ ]:
print(conversation_with_summary.prompt.template)

Identificamos que el prompt que usemos tiene que tener dos inputs: history e input. Con ello podemos diseñar nuestro prompt.

In [ ]:
from langchain.prompts.prompt import PromptTemplate

plantilla_chilena = """
La siguiente es una conversación entre un humano y una inteligencia artificial, compadre.
Esta IA, cachai, es un asistente de ventas de autos, bien metido en el tema.
Si la IA no cachara alguna respuesta, no va a tener atados pa' decir que no sabe, ah.
La IA va al hueso, sin rodeos. Pregunta directamente qué necesita con lo referente al auto.
La IA habla con una forma popular chilena y parecida a memes.

Conversación actual:
{history}
Humano: {input}
IA:
"""

PLANTILLA_CHILENA_CONVERSACION_RESUMEN = PromptTemplate(
    input_variables=["history", "input"], template=plantilla_chilena
)

In [ ]:
conversation_with_summary_chileno = ConversationChain(
    llm=llm,
      memory=memory,
    verbose=True,
    prompt=PLANTILLA_CHILENA_CONVERSACION_RESUMEN
)

In [ ]:
conversation_with_summary_chileno.predict(
    input="¿Oli como estai?"
    )

In [ ]:
conversation_with_summary_chileno.predict(
    input="Quiero comprar un auto, ya!"
    )

In [ ]:
conversation_with_summary_chileno.predict(
    input="Quiero comprar un auto, ya!"
    )

## 2.8 Entity memory - Guarda las entidades clave en la memoria

La memoria de entidades se comporta de manera diferente a los tipos anteriores. Esta estrategia de memoria se centra en reconocer y recordar entidades específicas mencionadas en la conversación. Esto es particularmente útil para chatbots que necesitan extraer y entender información clave, como nombres de personas, identificadores de productos y otros detalles importantes.

In [ ]:
from langchain.chains.conversation.memory import ConversationEntityMemory
from langchain.chains.conversation.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE

En cualquier momento podemos editar el template ingresado originalmente al modelo. La clave es recordar que es simplemente un objeto de tipo PromptTemplate (nota que el prompt cambia para cada diferente tipo de memoria) pero que tiene diferentes `input_variables` de acuerdo a cómo funciona el estilo de memoria. Es siempre util ver el template que se está utilizando para ver estas `input_variables` antes de crear nuestro prompio prompt template para este estilo de memoria.

Por ejemplo para la Entity Memory Conversation tenemos que usar tres input_variables: input_variables=['entities', 'history', 'input']

Aquí creamos nuestro propio prompt con un poco de sabor Chilango (persona nacida en la Ciudad de México).

In [ ]:
from langchain.prompts.prompt import PromptTemplate

template = """
Eres un asistente de ventas para una empresa de máquinas de micheladas,

Solo estás diseñado para (1) buscar resolver el problema con la máquina de micheladas, y si el cliente no lo logra, (2) agendar la visita de un técnico especializado.

Si el cliente no ha logrado arreglar la máquina, entonces pregunta si quiere agendar una visita con el técnico. Si el cliente quiere agendar una visita con el técnico entonces debe dejar su número de celular y dirección.

También cuentas con acceso a información personalizada proporcionada por el humano en la sección de Contexto a continuación.

Estás aquí para ayudar, siempre con la chispa y el carácter de alguien nacido en Tepito, México.

Es clave que preguntes la fecha en qué compraron la máquina, su número de garantía y quién los atendió. Pregunta siempre si se logro resolver el problema o duda del cliente.

Contexto:
{entities}

Conversación actual:
{history}

Última línea:
Humano: {input}

Tú (nacido en Tepito):
"""



In [ ]:
llm = OpenAI(model_name='text-davinci-003',
             temperature=0,
             max_tokens = 256)

Comenzamos la conversación:

In [ ]:
from pprint import pprint

